## Intro


On va essayer de comprendre comment fonctionne un module de torch

In [1]:
%reset -f

Lisons la docstring:

In [2]:
import torch
torch.nn.Module

In [3]:
import numpy as np

## Préliminaire de POO

### Classe

In [4]:
class Toto:
    """Ici c'est la docstring de la classe Toto"""
    def __init__(self,a):

        self.a=a
        self.b="Bou"

In [5]:
toto=Toto(3)
toto.__class__

In [6]:
toto.__class__.__name__

In [7]:
def myFunc():
    pass
myFunc.__name__

In [8]:
type(toto) is toto.__class__

In [9]:
type(toto) is Toto

### Classe et sur-classe

In [10]:
class Bou(Toto):
    def __init__(self):
        self.k=6

bou=Bou()
bou.__class__.__mro__

### attributs

In [11]:
toto=Toto(3)
print(toto.__dict__)

Attention, `__dict__` ne donne que les attributs propres à la classe.

In [12]:
bou=Bou()
print(bou.__dict__)

## Construisons notre lib

### Une classe imitant `torch.nn.Parameter`

In [13]:
class Parameter():
    def __init__(self,weights:torch.Tensor):
        self.weights=weights

In [14]:
param=Parameter(torch.zeros([5,5]))
param.weights.shape

### Une classe imitant `torch.nn.Module`

In [15]:
class Module():
    def __init__(self):
        self.training=True


    def get_children(self):
        res=[]
        for attribute in self.__dict__.values():
            if Module in attribute.__class__.__mro__:
                res.append(attribute)
        return res


    #parcours récursif de l'arbre des sous-modules
    def get_descendants(self):
        res=[]
        for child in self.get_children():
            res.append(child)
            res.extend(child.get_descendants())
        return res

    def get_me_and_descendants(self):
        res=[self]
        res.extend(self.get_descendants())
        return res


    def state_dict(self):
        res={}
        count=0
        for submodule in self.get_me_and_descendants():
            for attribute in submodule.__dict__.values():
                if type(attribute)==Parameter:
                    res[f"param_{count}"]= attribute.weights
                    count+=1
        return res

    def parameters(self):
        return self.state_dict().values()


    def __call__(self,X):
        return self.forward(X)


    """Les méthodes suivantes agissent sur tous les sous-modules"""

    def train():
        for submodule in self.get_family():
            submodule.training=True
    def eval():
        for submodule in self.get_family():
            submodule.training=False


    def to(self,arg):
        for param in self.parameters():
            param.weights.to(arg)



    """A la mode Tensorflow"""

    @property
    def trainable_variables(self):
        return self.parameters()

#### ♡♡♡

***A vous:*** Comment est-ce que vous pourriez-écrire la méthode `get_me_and_descendants` directement, sans passer par une méthode `get_descendants` ?

Solution:


    def get_me_and_descendants(self):
        res=[self]

        ...


### Tests

In [17]:
class ALayer(Module):
    def __init__(self,size):
        self.weights= Parameter(torch.zeros([size,size]))
        self.bias= Parameter(torch.zeros([size]))


class ABlock(Module):
    def __init__(self):
        self.layer_1=ALayer(2)
        self.layer_2=ALayer(3)


class AModel(Module):
    def __init__(self):
        self.proper_param=Parameter(torch.tensor(3.))
        self.block_1=ABlock()
        self.bloc_2=ABlock()
        self.a=7


model=AModel()
for module in model.get_me_and_descendants():
    print(module)

In [19]:
for p in model.parameters():
    print(p.shape)

In [20]:
for name,weight in model.state_dict().items():
    print(f"{name}:{weight.shape}")

### Tensorflow + keras


En tensorflow+keras c'est tout pareille sauf que l'équivalent d'un module c'est `tf.keras.Model`


Notez que la méthode `.parameters()` est remplacé par la property `.trainable_parameters`: c'est une méthode de type `getter` qui s'appelle sans parenthèse




In [21]:
for p in model.trainable_variables:
    print(p.shape)

### En savoir plus sur les properties

On peut aussi définir des setter:

In [22]:
class Man:
    def __init__(self):
        self._private_age=None

    @property
    def age(self):
        if self._private_age is None:
            raise Exception("l'age n'a pas encore été renseigné")
        return self._private_age

    @age.setter
    def age(self,value):
        value=int(value)
        self._private_age=value


man=Man()
man.age=18.5
print(man.age)

## A quoi sert l'attribut `training`?

In [23]:
drop_layer=torch.nn.Dropout(0.2)
drop_layer.training

In [24]:
A=torch.ones([1,3,5])
drop_layer(A)

#### ♡

A quoi correspond le 1.25?

In [26]:
drop_layer.eval()
drop_layer.training

In [27]:
drop_layer(A)

## Des noms automatiques

In [28]:
class Block(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.layerToto=torch.nn.Linear(5,5)
        self.layerBou=torch.nn.Linear(5,5)

In [29]:
class ModelTorch(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.block1=Block()
        self.block2=Block()

model=ModelTorch()

for k,v in model.state_dict().items():
    print(k,v.shape)

In [30]:
model.block1.layerToto.bias

***A vous:***

* Créez un layer `Linear` dans notre lib.
* Créez un layer `DropOut` dans notre lib (il faudra utiliser l'attribut `training`)
* Modifiez notre classe `Module` pour que des noms par défaut soient sympa, comme pour la vraie classe `torch.nn.Module`


#### ♡♡♡♡♡♡

In [33]:
class ALayer(Module):
    def __init__(self,size):
        self.weights= Parameter(torch.zeros([size,size]))
        self.bias= Parameter(torch.zeros([size]))


class ABlock(Module):
    def __init__(self):
        self.layer_1=ALayer(2)
        self.layer_2=ALayer(3)


class AModel(Module):
    def __init__(self):
        self.proper_param=Parameter(torch.tensor(3.))
        self.block_1=ABlock()
        self.bloc_2=ABlock()
        self.a=7


model=AModel()
for module in model.get_me_and_descendants():
    print(module.name)

In [33]:
#--- To keep following outputs, do not run this cell! ---

root
root.block_1
root.block_1.layer_1
root.block_1.layer_2
root.bloc_2
root.bloc_2.layer_1
root.bloc_2.layer_2


In [34]:
for k,v in model.state_dict().items():
    print(k,":",v.shape)

In [34]:
#--- To keep following outputs, do not run this cell! ---

root.proper_param : torch.Size([])
root.block_1.layer_1.weights : torch.Size([2, 2])
root.block_1.layer_1.bias : torch.Size([2])
root.block_1.layer_2.weights : torch.Size([3, 3])
root.block_1.layer_2.bias : torch.Size([3])
root.bloc_2.layer_1.weights : torch.Size([2, 2])
root.bloc_2.layer_1.bias : torch.Size([2])
root.bloc_2.layer_2.weights : torch.Size([3, 3])
root.bloc_2.layer_2.bias : torch.Size([3])


In [36]:
class ALayer(Module):
    def __init__(self,size):
        self.weights= Parameter(torch.zeros([size,size]))
        self.bias= Parameter(torch.zeros([size]))

class AModel(Module):
    def __init__(self):
        self.proper_param=Parameter(torch.tensor(3.))
        self.layer_1=ALayer(3)

model=AModel()
for module in model.get_me_and_descendants():
    print(module.name)

In [36]:
#--- To keep following outputs, do not run this cell! ---

root
root.layer_1


In [37]:
for k,v in model.state_dict().items():
    print(k,":",v.shape)